In [5]:
import requests
import pandas as pd
import logging
from datetime import datetime, timedelta, timezone
from OddsJamClient import OddsJamClient


In [6]:
api_key_path = 'api_key.txt'

def load_api_key(path):
    with open(path, 'r') as file:
        return file.readline().strip()

api_key = load_api_key(api_key_path)


In [39]:
def fetch_sports_markets(sport, league, api_key, sportsbook=['Pinnacle']):
    url = "https://api.opticodds.com/api/v3/markets"
    params = {
        'sport': sport,
        'league': league,
        'key':api_key
    }
    
    # Optional: Specify a sportsbook if required
    if sportsbook:
        params['sportsbook'] = sportsbook

    headers = {'Authorization': f'Bearer {api_key}'}
    
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        # Assuming the data is formatted in a list of markets, convert it into a DataFrame
        markets_df = pd.DataFrame(data['data'])
        return markets_df
    else:
        print(f"Failed to fetch markets: {response.status_code} - {response.text}")
        return pd.DataFrame()
def american_to_decimal(odds):
    if odds > 0:
        return (odds / 100) + 1
    else:
        return (100 / abs(odds)) + 1

def implied_probability(decimal_odds):
    return 1 / decimal_odds

def remove_vig(implied_prob_a, implied_prob_b):
    no_vig_prob_a = implied_prob_a / (implied_prob_a + implied_prob_b)
    return no_vig_prob_a
def adjust_for_vig(implied_prob, vig_reduction=0.02):
    return implied_prob / (1 + vig_reduction)

def calculate_ev(true_prob, odds, stake=100):
    # Convert American odds to decimal odds
    decimal_odds = american_to_decimal(odds)
    
    # Calculate profit if win
    profit_if_win = (decimal_odds - 1) * stake
    
    # Calculate EV using the formula (True Probability * Profit if Win) - (Loss Probability * Stake)
    loss_prob = 1 - true_prob
    ev = (true_prob * profit_if_win) - (loss_prob * stake)
    
    return ev

def power_method_de_vig(prob, power=0.98):  # Use a more moderate power value
    return prob ** power

def find_plus_ev_bets(df, thresh):
    caesars_df = df[df['Sportsbook'] == 'Caesars']
    pinnacle_df = df[df['Sportsbook'] == 'Pinnacle']
    betmgm_df = df[df['Sportsbook'] == 'BetMGM']

    betmgm_df = betmgm_df.rename(columns={'Odds': 'Odds_betmgm', 'line': 'line_betmgm'})

    merged_df = pd.merge(caesars_df, pinnacle_df, on=['Game ID', 'Bet Name', 'Market Name'], suffixes=('_caesars', '_pinnacle'))

    merged_df = pd.merge(merged_df, betmgm_df, on=['Game ID', 'Bet Name', 'Market Name'], how='left')

    
    merged_df['decimal_odds_caesars'] = merged_df['Odds_caesars'].apply(american_to_decimal)
    merged_df['decimal_odds_pinnacle'] = merged_df['Odds_pinnacle'].apply(american_to_decimal)
    merged_df['decimal_odds_betmgm'] = merged_df['Odds_betmgm'].apply(american_to_decimal)
    
    merged_df['implied_prob_caesars'] = merged_df['decimal_odds_caesars'].apply(implied_probability)
    merged_df['implied_prob_pinnacle'] = merged_df['decimal_odds_pinnacle'].apply(implied_probability)
    merged_df['implied_prob_betmgm'] = merged_df['decimal_odds_betmgm'].apply(implied_probability)

    merged_df['true_prob_pinnacle'] = merged_df['implied_prob_pinnacle'].apply(lambda prob: adjust_for_vig(prob, 0.02))  # Assuming 2% is the vig you want to remove

    
    merged_df['EV_caesars'] = merged_df.apply(
        lambda row: calculate_ev(row['true_prob_pinnacle'], row['Odds_caesars']) 
        if row['true_prob_pinnacle'] > row['implied_prob_caesars']  
        else -1, 
        axis=1)

    merged_df['EV_betmgm'] = merged_df.apply(
        lambda row: calculate_ev(row['true_prob_pinnacle'], row['Odds_betmgm']) 
        if row['true_prob_pinnacle'] > row['implied_prob_betmgm']  
        else -1, 
        axis=1)

    positive_ev_bets = merged_df[(merged_df['EV_caesars'] > thresh) | (merged_df['EV_betmgm'] > thresh)]
    
    return positive_ev_bets

def fetch_game_data(game_ids, markets, sportsbooks, include_player_name=True):
    URL = "https://api-external.oddsjam.com/api/v2/game-odds"
    API_KEY = api_key
    all_data = []

    for chunk in [game_ids[i:i + 5] for i in range(0, len(game_ids), 5)]:
        for sportsbook in sportsbooks:
            params = {
                'key': API_KEY,
                'sportsbook': sportsbook,
                'game_id': chunk,
                'market_name': markets
            }
            response = requests.get(URL, params=params)
            if response.status_code == 200:
                data = response.json().get('data', [])
                all_data.extend(data)
            else:
                logging.error("Error fetching data: %s - %s", response.status_code, response.text)
    rows = []
    for game_data in all_data:
        home_team = game_data.get('home_team', 'Unknown')
        away_team = game_data.get('away_team', 'Unknown')
        odds_list = game_data.get('odds', [])
            
        for item in odds_list:
            market_name = item.get('market_name', '')
            sportsbook_name = item.get('sports_book_name', sportsbook)
            if market_name not in markets:
                continue

            row = {
                'Game ID': game_data.get('id', 'Unknown'),
                "Game Name": home_team+" vs "+away_team,
                "Bet Name":item.get('name',None),
                'Market Name': market_name,
                'Sportsbook': sportsbook_name,
                'line':item.get('bet_points',None),
                'Odds': item.get('price', None),
            }

            if include_player_name:
                player_name = item.get('selection', 'Unknown')
                row['Player Name'] = player_name

            rows.append(row)

    return pd.DataFrame(rows)


### MLB

In [40]:
def get_todays_game_ids():
    
    Client = OddsJamClient(api_key)
    Client.UseV2()
    
    # Get games for the league
    GamesResponse = Client.GetGames(league='mlb')
    Games = GamesResponse.Games
    
    # Filter games based on today's date
    games_data = [{'game_id': game.id, 'start_date': game.start_date} for game in Games]
    games_df = pd.DataFrame(games_data)
    games_df['start_date'] = pd.to_datetime(games_df['start_date'])
    #today = datetime.now().date()
    #todays_games = games_df[games_df['start_date'].dt.date == today]
    
    return games_df['game_id'].tolist()

    


def fetch_player_props(game_ids):
    player_markets = [
        "Player Hits Allowed", "Player Strikeouts", "Player Home Runs",
        "Player Bases", "Player Home Runs Yes/No", "Player Outs", "Player Earned Runs"
    ]
    return fetch_game_data(game_ids, player_markets, ['Pinnacle','BetMGM','Caesars'], include_player_name=True)

def fetch_game_props(game_ids):
    game_markets = [
        "Team Total", "1st Half Team Total", "1st Half Moneyline", "Moneyline",
        "No Runs First Inning", "1st Inning Moneyline 3-Way", "1st Half Run Line",
        "Total Runs", "1st Inning Total Runs", "1st Half Total Runs", "Run Line",
        "1st Inning Run Line", "Total Hits + Runs + Errors"
    ]
    return fetch_game_data(game_ids, game_markets, ['Pinnacle','BetMGM','Caesars'], include_player_name=False)


In [52]:
todays_ids = get_todays_game_ids()
pdf = fetch_player_props(game_ids=todays_ids)
#gdf = fetch_game_props(todays_ids)

In [53]:
evdf = find_plus_ev_bets(pdf,thresh=5)
betmgmdf = evdf[['Game ID','Bet Name',"Game Name",'Market Name','line_betmgm','Odds_betmgm','line_pinnacle','Odds_pinnacle','EV_betmgm']]
betmgmdf = betmgmdf.sort_values('EV_betmgm',ascending=False)


In [65]:
#betmgmdf

In [63]:
#betmgmdf.to_csv('mlb_player_props_9_20.csv',index=False,header=True)

### NCAAF

In [56]:
## NCAA Football
def get_ncaa_game_ids():
    # Endpoint
    endpoint = "https://api-external.oddsjam.com/api/v2/games"

    # Current week date range
    today = datetime.now(timezone.utc)
    start_date = today.isoformat()


    # Calculate the upcoming Monday or today if it's Monday
    days_until_monday = (7 - today.weekday()) % 7
    monday = today + timedelta(days=days_until_monday)
    
    # Set time to 11:59 pm and adjust for CST (UTC-6)
    end_date_cst = (monday + timedelta(days=1)).replace(hour=5, minute=59, second=59, microsecond=0, tzinfo=timezone.utc)
    # Parameters
    params = {
        "key": api_key,
        "sport": "football",
        "league": "NCAAF",
        "start_date_after": start_date,
        "start_date_before": end_date_cst.isoformat()
    }

    # Make the request
    response = requests.get(endpoint, params=params)

    # Check response
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None

    data = response.json()
    game_ids = [game['id'] for game in data['data']]
    return game_ids


In [58]:
# Example usage
api_key = api_key
ncaaf_markets = fetch_sports_markets('football', 'NCAAF', api_key)
ncaa_ids = get_ncaa_game_ids()
ncaaf_markets = ncaaf_markets['name'].tolist()
ncaaf_df = fetch_game_data(ncaa_ids,ncaaf_markets,['Pinnacle','BetMGM','Caesars'])
ncaaf_df_ev = find_plus_ev_bets(ncaaf_df,thresh=5)
betmgmdf_ncaa = ncaaf_df_ev[['Game ID','Bet Name',"Game Name",'Market Name','line_betmgm','Odds_betmgm','line_pinnacle','Odds_pinnacle','EV_betmgm']]
betmgmdf_ncaa = betmgmdf_ncaa.sort_values('EV_betmgm',ascending=False)


In [62]:
#betmgmdf_ncaa

In [64]:
#betmgmdf_ncaa.to_csv('ncaaf_game_bets_9_20.csv',index=False,header=True)